In [2]:
import os

In [3]:
os.listdir("Rice_Image_Dataset")


['Arborio', 'Basmati', 'Ipsala', 'Jasmine', 'Karacadag']

In [4]:
import os

original_path = "Rice_Image_Dataset"

for class_name in os.listdir(original_path):
    full_path = os.path.join(original_path, class_name)
    if os.path.isdir(full_path):
        print(f"{class_name}: {len(os.listdir(full_path))} images")


Arborio: 15000 images
Basmati: 15000 images
Ipsala: 15000 images
Jasmine: 15000 images
Karacadag: 15000 images


In [6]:
import os
import shutil
import random

# Paths
SOURCE_DIR = "Rice_Image_Dataset"
DEST_DIR = "small_split_dataset"
TRAIN_SIZE = 800
TEST_SIZE = 200

# Make base destination folders
for subset in ['train', 'test']:
    os.makedirs(os.path.join(DEST_DIR, subset), exist_ok=True)

# Loop through each class
for class_name in os.listdir(SOURCE_DIR):
    class_path = os.path.join(SOURCE_DIR, class_name)
    
    # Skip if not a folder
    if not os.path.isdir(class_path):
        continue
    
    # List and shuffle image files
    images = os.listdir(class_path)
    random.shuffle(images)
    
    # Select only 1000 images
    selected_images = images[:TRAIN_SIZE + TEST_SIZE]
    train_imgs = selected_images[:TRAIN_SIZE]
    test_imgs = selected_images[TRAIN_SIZE:]
    
    # Create class folders
    train_class_dir = os.path.join(DEST_DIR, 'train', class_name)
    test_class_dir = os.path.join(DEST_DIR, 'test', class_name)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)
    
    # Copy train images
    for img in train_imgs:
        shutil.copy(os.path.join(class_path, img), os.path.join(train_class_dir, img))
    
    # Copy test images
    for img in test_imgs:
        shutil.copy(os.path.join(class_path, img), os.path.join(test_class_dir, img))


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# New dataset paths
train_path = "small_split_dataset/train"
test_path = "small_split_dataset/test"

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

# Training data generator with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Test data generator (only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load training data
train_set = train_datagen.flow_from_directory(
    train_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Load test data
test_set = test_datagen.flow_from_directory(
    test_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 4000 images belonging to 5 classes.
Found 1000 images belonging to 5 classes.


In [8]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load base MobileNet
base_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)

# Compile the model
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_set, validation_data=test_set, epochs=10)


Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 244s 2s/step - accuracy: 0.8417 - loss: 0.4849 - val_accuracy: 0.9620 - val_loss: 0.0938
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 126s 1s/step - accuracy: 0.9655 - loss: 0.0997 - val_accuracy: 0.9740 - val_loss: 0.0702
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 126s 1s/step - accuracy: 0.9706 - loss: 0.0893 - val_accuracy: 0.9660 - val_loss: 0.1023
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 125s 1000ms/step - accuracy: 0.9713 - loss: 0.0864 - val_accuracy: 0.9680 - val_loss: 0.0808
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 125s 998ms/step - accuracy: 0.9825 - loss: 0.0522 - val_accuracy: 0.9800 - val_loss: 0.0466
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 126s 1s/step - accuracy: 0.9800 - loss: 0.0536 - val_accuracy: 0.9800 - val_loss: 0.0561
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 125s 998ms/step - accuracy: 0.9843 - loss: 0.0438 - val_accuracy: 0.9240 - val_loss: 0.2328
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 125s 997ms/step - accuracy: 0.9715 - loss: 0.07